In [3]:
# 📦 Import
import pandas as pd
from pymongo import MongoClient
from scdv.utilities.atomic_dataframe import DataFrameAtomic
from scdv.utilities.mongodb_handler import atomic_services_collection

In [12]:
pd.set_option('display.max_columns', None)  # mostra tutte le colonne
pd.set_option('display.max_rows', None)     # mostra tutte le righe
pd.set_option('display.max_colwidth', None) # mostra tutto il contenuto delle celle
pd.set_option('display.expand_frame_repr', False)  # evita il wrapping su più righe

In [4]:
# 📥 Recupera tutti i documenti
docs = list(atomic_services_collection.find({}))
print(f"Trovati {len(docs)} atomic services.")

Trovati 5 atomic services.


In [6]:
# 🔄 Processa tutti i documenti
flatted_dfs = []

for doc in docs:
    payload = {
        'diagram_id': doc['diagram_id'],
        'task_id': doc['task_id'],
        'name': doc['name'],
        'atomic_type': doc['atomic_type'],
        'method': doc['method'],
        'url': doc['url'],
        'owner': doc['owner'],
        'input_params': doc['dataframe_serialized'].get('input_params', []),
        'output_params': doc['dataframe_serialized'].get('output_params', [])
    }
    
    df_atomic = DataFrameAtomic(payload)
    df_atomic.create_main_dataframe()
    df_atomic.create_io_dataframes()
    flatted_df = df_atomic.get_flatted_wide_table()
    
    flatted_dfs.append(flatted_df)

In [7]:
# 🛠 Combina in un unico DataFrame
all_flatted = pd.concat(flatted_dfs, ignore_index=True)
print(f"✅ Combinati in un DataFrame di shape: {all_flatted.shape}")

✅ Combinati in un DataFrame di shape: (5, 33)


In [13]:
# 🔍 Mostra le prime righe
all_flatted.head()

,diagram_id,task_id,name,atomic_type,method,url,owner,input_1,input_2,input_3,input_1_type,input_2_type,input_3_type,output_1,output_2,output_3,output_1_type,output_2_type,output_3_type,input_4,input_4_type,output_4,output_5,output_4_type,output_5_type,input_5,input_6,input_5_type,input_6_type,output_6,output_7,output_6_type,output_7_type
0,6878fe9339a29517d132694a,Activity_0r3yt5t,prova_df,collect,GET,/make_prova,prova,a,b,c,String,String,String,d,e,f,String,String,String,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6878fe9339a29517d132694a,Activity_1vmqw1f,prova_df_2,process&monitor,GET,/make_post,prova,a,b,2,String,String,Int,b,c,stringa,String,String,String,3.2,Float,2.0,4.5,Float,Float,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,687900f0aea7197c4b4915b4,Activity_04lnxq6,prova_flatted_df,collect,GET,/get_flatted,prova_flatted,a,b,c,String,String,String,mela,pera,nano,String,String,String,2,Int,1,2.7,Int,Float,stringa,3.5,String,Float,NaN,NaN,NaN,NaN
3,68790233851f12b16a591983,Activity_0xdmv4b,flatted_wide_table,collect,GET,/make_flatted_wide_table,pippo,a,b,2.7,String,String,Float,b,c,d,String,String,String,3.0,Float,e,f,String,String,NaN,NaN,NaN,NaN,4.3,5,Float,Int
4,687902c802536edd5a83a6d7,Activity_14b3r8f,flatted_with_types,collect,GET,/make_flatted_with_types,pippo,stringa,2,3.0,String,Int,Float,2.35,3,7a,Float,Int,String,b,String,stringa,NaN,String,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
all_flatted.columns

Index(['diagram_id', 'task_id', 'name', 'atomic_type', 'method', 'url',
       'owner', 'input_1', 'input_2', 'input_3', 'input_1_type',
       'input_2_type', 'input_3_type', 'output_1', 'output_2', 'output_3',
       'output_1_type', 'output_2_type', 'output_3_type', 'input_4',
       'input_4_type', 'output_4', 'output_5', 'output_4_type',
       'output_5_type', 'input_5', 'input_6', 'input_5_type', 'input_6_type',
       'output_6', 'output_7', 'output_6_type', 'output_7_type'],
      dtype='object')

In [11]:
# 🔎 Controlla tipi di atomic_type presenti
all_flatted['atomic_type'].value_counts()

atomic_type
collect            4
process&monitor    1
Name: count, dtype: int64